In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
import os
import pickle
from feature_extraction import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
TRAIN_PATH = "../data/processed/train2/"
# TEST_PATH = "../data/processed/test/"
REF_PATH = "../data/processed/reference/"
WRITE_PATH = "../data/models/"

In [3]:
def setup_df(PATH):
    all_files = os.listdir(PATH)
    df = pd.DataFrame(all_files, columns=['file'])
    df['speaker'] = df['file'].apply(lambda x: int(x.split('.')[0].split('-')[-1]))
    # 0 for neutral, 1 for emotional
    df['speech_type'] = df['file'].apply(lambda x: 0 if int(x.split('.')[0].split('-')[2])==1 else 1)
    df['F0_contour'] = df['file'].apply(lambda x: get_F0_contour(PATH+x))
    return df

In [4]:
def get_audio_features(df, norm=False):
    if norm==False:
        df['SQ25'] = df['F0_contour'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour'].apply(get_SQ75)
        df['IDR'] = df['F0_contour'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour'].apply(get_sdmedian)
    else:
        df['SQ25'] = df['F0_contour_norm'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour_norm'].apply(get_SQ75)
        df['IDR'] = df['F0_contour_norm'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour_norm'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour_norm'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour_norm'].apply(get_sdmedian)
    
    df['SVMeanRange'] = df['voiced_segments'].apply(get_voiced_segment_range)
    df['SVMaxCurv'] = df['voiced_segments'].apply(get_max_voiced_curvature)
    
    return df

In [5]:
ref_df = setup_df(REF_PATH)
ref_df = get_audio_features(ref_df)
temp_df = setup_df(TRAIN_PATH)
# test_df = setup_df(TEST_PATH)

## Training the GMMs on the Reference Data

In [16]:
trained_GMMs = {}
features = [ 'SQ25', 'SQ75','F0_median', 'sdmedian', 'IDR', 'SVMeanRange','SVMaxCurv']
# features = [ 'SQ75','IDR', 'SVMeanRange','SVMaxCurv']

for feature in features:
    gmm = GaussianMixture(n_components=2)
    gmm.fit(ref_df[feature].values.reshape(-1,1))
    trained_GMMs[feature] = gmm

## IFN Algo

In [17]:
ref_df['F0_contour_sum'] = ref_df['F0_contour'].apply(sum)

In [18]:
ref_df['F0_contour_length'] = ref_df['F0_contour'].apply(len)

In [19]:
ref_df['F0_contour_mean'] = ref_df['F0_contour_sum']/ref_df['F0_contour_length']

In [20]:
avg_F0_ref = np.sum(ref_df['F0_contour_mean']) / 24

In [21]:
def stratified_sample_df(df):
    per_speaker_neutral_number = df[df['speech_type']==0]['speaker'].value_counts().values[0]
    sampled_df = []
    sampled_df.append(df[df['speech_type']==0])
    speakers = df['speaker'].unique()
    for speaker in speakers:
        speaker_df = df[df['speaker'] == speaker]
        speaker_emotional_df= speaker_df[speaker_df['speech_type']==1]
        sampled_df.append(speaker_emotional_df.sample(n=per_speaker_neutral_number))
    return pd.concat(sampled_df).reset_index(drop=True)

In [22]:
def infer_GMM(df):
    infered = np.zeros(shape=(1,len(features)))
    for i in range(len(features)):
        res = trained_GMMs[features[i]].score_samples([[df[features[i]]]])
        infered[:,i] = res
    return infered[0]

In [23]:
def get_changed_labels(neutral_list, emotional_list, row):
    if row['file'] in neutral_list:
        return 0
    else:
        return 1

In [24]:
def get_S_s_F0(F0_ref, df):
#     grouped_df_neu = df[df['speech_type']==0].groupby(['speaker']).sum().reset_index(level=0)
#     grouped_df_neu = grouped_df_neu[['speaker','F0_contour_sum','F0_contour_length']]
#     grouped_df_neu['F0_s_neu'] = grouped_df_neu['F0_contour_sum'] / grouped_df_neu['F0_contour_length']
#     grouped_df_neu['S_s_F0'] = F0_ref / grouped_df_neu['F0_s_neu']

    df_neu = df[df['changed_speech_type']==0]
    speakers = df['speaker'].unique()
    grouped_df_neu = {}
    UPPER_CAP = 100
    LOWER_CAP = 0.00001
    for speaker in speakers:
        speaker_df_neu = df_neu[df_neu['speaker']==speaker]
        speaker_mean = (speaker_df_neu['F0_contour_sum']/speaker_df_neu['F0_contour_length']).mean()
        if F0_ref/speaker_mean > UPPER_CAP:
            grouped_df_neu[speaker] = UPPER_CAP
        elif F0_ref/speaker_mean < LOWER_CAP:
            grouped_df_neu[speaker] = LOWER_CAP
        else:
            grouped_df_neu[speaker] = F0_ref/speaker_mean
    
    
        
        
    
#     return grouped_df_neu[['speaker','S_s_F0']].set_index('speaker').to_dict()['S_s_F0']
    return grouped_df_neu

def get_normalised_df(df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0):
    df_norm = S0_func(avg_F0_ref, df)
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df
        

def get_normalised_df_infer(df, df_norm):
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df

In [25]:
CLASSIFICATION_THRESHOLD = 0.5

In [26]:
def get_pred_labels(df):
    
    grouped_sampled_df = df.groupby('speaker')
    neutral = []
    emotional = []

    for name, group in grouped_sampled_df:
        neu_result = group[group['predicted_likelihood'] >= CLASSIFICATION_THRESHOLD]
        emo_result = group[group['predicted_likelihood'] < CLASSIFICATION_THRESHOLD]

        total = group.shape[0]
        to_add = int(np.ceil(0.2 * total)) - neu_result.shape[0]
        converted_neu_add = neu_result
        if to_add > 0:
            emo_result_sort = emo_result.sort_values('predicted_likelihood', ascending=False)
            converted_neu_add = emo_result_sort.head(to_add)
            emo_result.drop(converted_neu_add.index, inplace=True)

        neutral.extend(converted_neu_add['file'].tolist())
        emotional.extend(emo_result['file'].tolist()) 

    df['changed_speech_type'] = df.apply(lambda x: \
                                         get_changed_labels(neutral,emotional,x), axis=1) 
    
    return df
    

In [27]:
def get_stopping_criteria(df, count):
    df = get_pred_labels(df)
    
    if count==0:
        return df, 1000000

    else:
        changed_dict = (df['prev_changed_speech_type'] != df['changed_speech_type']).\
                        value_counts().to_dict()

        if True not in changed_dict.keys():
            changed_dict[True] = 0
            epsilon = changed_dict[True]/changed_dict[False]
        elif False not in changed_dict.keys():
            epsilon = 1000000
        else:
            epsilon = changed_dict[True]/changed_dict[False]
    
    return df, epsilon

In [28]:
EXPERIMENT_TYPE = 'norm_original_ldc_0.5'

In [29]:
# for every epoch
epoch = 0

ITERATION_CLF_REPORT = []
ITERATION_FILE_CHANGE = []
LDC_CLFS = []


for iterations in range(400):

    sampled_df = stratified_sample_df(temp_df)
    
    train_df, test_df = train_test_split(sampled_df, test_size = 0.33, stratify = sampled_df[['speaker','speech_type']])
    
    train_df = get_audio_features(train_df)
    test_df = get_audio_features(test_df)
    
    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)

    count = 0

    # for 400 iterations
    sampled_df_norm = None
    ldc_clf = None
    epsilon = 1000000
    max_iters = 1000
    
    ldc_clf = LinearDiscriminantAnalysis(solver='lsqr')
    
    print('*************************************************') 
    print('ITERATION NUMBER - 'iterations)
    print('*************************************************') 

            
    for stage in ['train', 'test']:

        print('----------------------------------------')   
        print('Stage - ', stage)


        if stage == 'train':
            
            ITERATION_FILE_CHANGE_ELEM = []
            
            while epsilon > 0.05 and count < max_iters:
        
                print('=========================================')   
                

                if count ==0 :
                    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)
                    train_df = get_audio_features(train_df, norm=False)
                else:
                    train_df['F0_contour_sum'] = train_df['F0_contour_norm'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour_norm'].apply(len)
                    # Change above to F0_contour_norm when norm=True
                    train_df = get_audio_features(train_df, norm=True)
                        
                train_df['inferred'] = train_df.apply(infer_GMM,axis=1)
                
                ldc_clf.fit(np.array(train_df['inferred'].tolist()),train_df['speech_type'].values)

                train_df['predicted_likelihood'] = ldc_clf.predict_proba(np.array(train_df['inferred'].tolist()))[:,0]

                train_df, epsilon = get_stopping_criteria(train_df, count)
                
                print(count)
                print(epsilon)
                ITERATION_FILE_CHANGE_ELEM.append(epsilon)
                LDC_CLFS.append(ldc_clf)
                
                
                train_df['prev_changed_speech_type'] = train_df['changed_speech_type']
                                
                sampled_df_norm, train_df = get_normalised_df(train_df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0)                
                count+=1
                
            ITERATION_FILE_CHANGE.append(ITERATION_FILE_CHANGE_ELEM)

        else:
                sampled_df_test = test_df
                if count!=0:
                    _, sampled_df_test = get_normalised_df_infer(test_df, sampled_df_norm)
                if count == 0 :
                    sampled_df_test = get_audio_features(sampled_df_test, norm=False)
                else:
                    sampled_df_test = get_audio_features(sampled_df_test, norm=True)
                sampled_df_test['inferred'] = sampled_df_test.apply(infer_GMM,axis=1)

                sampled_df_test['predicted_likelihood'] = ldc_clf.predict_proba(np.array(sampled_df_test['inferred'].tolist()))[:,0]
#                 sampled_df_test['changed_speech_type'] = sampled_df_test['predicted_likelihood'].\
#                                                             apply(lambda x: 0 if x >=0.7 else 1)

                sampled_df_test = get_pred_labels(sampled_df_test)

                clf_report = classification_report(sampled_df_test['speech_type'],sampled_df_test['changed_speech_type'], output_dict=True)
                ITERATION_CLF_REPORT.append(clf_report)
                print(clf_report)
                
                
                
#         count+=1



*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.17073170731707318
2
0.06666666666666667
3
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.6, 'recall': 1.0, 'f1-score': 0.7499999999999999, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.3333333333333333, 'f1-score': 0.5, 'support': 24}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.8, 'recall': 0.6666666666666666, 'f1-score': 0.625, 'support': 48}, 'weighted avg': {'precision': 0.7999999999999999, 'recall': 0.6666666666666666, 'f1-score': 0.6249999999999999, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.10344827586206896
2
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precisio

1
0.17073170731707318
2
0.06666666666666667
3
0.07865168539325842
4
0.054945054945054944
5
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.5384615384615384, 'recall': 0.875, 'f1-score': 0.6666666666666667, 'support': 24}, '1': {'precision': 0.6666666666666666, 'recall': 0.25, 'f1-score': 0.36363636363636365, 'support': 24}, 'accuracy': 0.5625, 'macro avg': {'precision': 0.6025641025641025, 'recall': 0.5625, 'f1-score': 0.5151515151515151, 'support': 48}, 'weighted avg': {'precision': 0.6025641025641025, 'recall': 0.5625, 'f1-score': 0.5151515151515151, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.21518987341772153
2
0.11627906976744186
3
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.6486486486486487, 'recall': 1.0, 'f1-score': 0.7868852459016393, 's

{'0': {'precision': 0.5789473684210527, 'recall': 0.9166666666666666, 'f1-score': 0.7096774193548387, 'support': 24}, '1': {'precision': 0.8, 'recall': 0.3333333333333333, 'f1-score': 0.47058823529411764, 'support': 24}, 'accuracy': 0.625, 'macro avg': {'precision': 0.6894736842105263, 'recall': 0.625, 'f1-score': 0.5901328273244781, 'support': 48}, 'weighted avg': {'precision': 0.6894736842105263, 'recall': 0.625, 'f1-score': 0.5901328273244781, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.14285714285714285
2
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.5476190476190477, 'recall': 0.9583333333333334, 'f1-score': 0.696969696969697, 'support': 24}, '1': {'precision': 0.8333333333333334, 'recall': 0.20833333333333334, 'f1-score': 0.33333333333333337, 'support': 24}, 'accuracy': 0.5833333333333334, 

0
1000000
1
0.2972972972972973
2
0.352112676056338
3
0.11627906976744186
4
0.12941176470588237
5
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.5641025641025641, 'recall': 0.9166666666666666, 'f1-score': 0.6984126984126983, 'support': 24}, '1': {'precision': 0.7777777777777778, 'recall': 0.2916666666666667, 'f1-score': 0.42424242424242425, 'support': 24}, 'accuracy': 0.6041666666666666, 'macro avg': {'precision': 0.670940170940171, 'recall': 0.6041666666666666, 'f1-score': 0.5613275613275612, 'support': 48}, 'weighted avg': {'precision': 0.6709401709401709, 'recall': 0.6041666666666666, 'f1-score': 0.5613275613275612, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.24675324675324675
2
0.11627906976744186
3
0.06666666666666667
4
0.043478260869565216
----------------------------------------
Stage -  tes

0
1000000
1
0.10344827586206896
2
0.054945054945054944
3
0.03225806451612903
----------------------------------------
Stage -  test
{'0': {'precision': 0.5897435897435898, 'recall': 0.9583333333333334, 'f1-score': 0.7301587301587301, 'support': 24}, '1': {'precision': 0.8888888888888888, 'recall': 0.3333333333333333, 'f1-score': 0.48484848484848486, 'support': 24}, 'accuracy': 0.6458333333333334, 'macro avg': {'precision': 0.7393162393162394, 'recall': 0.6458333333333334, 'f1-score': 0.6075036075036075, 'support': 48}, 'weighted avg': {'precision': 0.7393162393162394, 'recall': 0.6458333333333334, 'f1-score': 0.6075036075036074, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.21518987341772153
2
0.06666666666666667
3
0.010526315789473684
----------------------------------------
Stage -  test
{'0': {'precision': 0.6216216216216216, 'recall': 0.9583333

...

0
1000000
1
0.09090909090909091
2
0.07865168539325842
3
0.054945054945054944
4
0.043478260869565216
----------------------------------------
Stage -  test
{'0': {'precision': 0.5384615384615384, 'recall': 0.875, 'f1-score': 0.6666666666666667, 'support': 24}, '1': {'precision': 0.6666666666666666, 'recall': 0.25, 'f1-score': 0.36363636363636365, 'support': 24}, 'accuracy': 0.5625, 'macro avg': {'precision': 0.6025641025641025, 'recall': 0.5625, 'f1-score': 0.5151515151515151, 'support': 48}, 'weighted avg': {'precision': 0.6025641025641025, 'recall': 0.5625, 'f1-score': 0.5151515151515151, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.2
2
0.07865168539325842
3
0.010526315789473684
----------------------------------------
Stage -  test
{'0': {'precision': 0.5, 'recall': 0.8333333333333334, 'f1-score': 0.625, 'support': 24}, '1': {'precision': 0.5, '

0
1000000
1
0.24675324675324675
2
0.10344827586206896
3
0.07865168539325842
4
0.010526315789473684
----------------------------------------
Stage -  test
{'0': {'precision': 0.55, 'recall': 0.9166666666666666, 'f1-score': 0.6874999999999999, 'support': 24}, '1': {'precision': 0.75, 'recall': 0.25, 'f1-score': 0.375, 'support': 24}, 'accuracy': 0.5833333333333334, 'macro avg': {'precision': 0.65, 'recall': 0.5833333333333333, 'f1-score': 0.53125, 'support': 48}, 'weighted avg': {'precision': 0.65, 'recall': 0.5833333333333334, 'f1-score': 0.5312499999999999, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.21518987341772153
2
0.07865168539325842
3
0.07865168539325842
4
0.043478260869565216
----------------------------------------
Stage -  test
{'0': {'precision': 0.6216216216216216, 'recall': 0.9583333333333334, 'f1-score': 0.7540983606557377, 'support

*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.10344827586206896
2
0.02127659574468085
----------------------------------------
Stage -  test
{'0': {'precision': 0.5526315789473685, 'recall': 0.875, 'f1-score': 0.6774193548387096, 'support': 24}, '1': {'precision': 0.7, 'recall': 0.2916666666666667, 'f1-score': 0.4117647058823529, 'support': 24}, 'accuracy': 0.5833333333333334, 'macro avg': {'precision': 0.6263157894736842, 'recall': 0.5833333333333334, 'f1-score': 0.5445920303605313, 'support': 48}, 'weighted avg': {'precision': 0.6263157894736842, 'recall': 0.5833333333333334, 'f1-score': 0.5445920303605313, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.14285714285714285
2
0.06666666666666667
3
0.06666666666666667
4
0.0434782

0
1000000
1
0.11627906976744186
2
0.054945054945054944
3
0.043478260869565216
----------------------------------------
Stage -  test
{'0': {'precision': 0.5789473684210527, 'recall': 0.9166666666666666, 'f1-score': 0.7096774193548387, 'support': 24}, '1': {'precision': 0.8, 'recall': 0.3333333333333333, 'f1-score': 0.47058823529411764, 'support': 24}, 'accuracy': 0.625, 'macro avg': {'precision': 0.6894736842105263, 'recall': 0.625, 'f1-score': 0.5901328273244781, 'support': 48}, 'weighted avg': {'precision': 0.6894736842105263, 'recall': 0.625, 'f1-score': 0.5901328273244781, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.23076923076923078
2
0.1566265060240964
3
0.043478260869565216
----------------------------------------
Stage -  test
{'0': {'precision': 0.5833333333333334, 'recall': 0.875, 'f1-score': 0.7000000000000001, 'support': 24}, '1': {'p

*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.17073170731707318
2
0.07865168539325842
3
0.03225806451612903
----------------------------------------
Stage -  test
{'0': {'precision': 0.5641025641025641, 'recall': 0.9166666666666666, 'f1-score': 0.6984126984126983, 'support': 24}, '1': {'precision': 0.7777777777777778, 'recall': 0.2916666666666667, 'f1-score': 0.42424242424242425, 'support': 24}, 'accuracy': 0.6041666666666666, 'macro avg': {'precision': 0.670940170940171, 'recall': 0.6041666666666666, 'f1-score': 0.5613275613275612, 'support': 48}, 'weighted avg': {'precision': 0.6709401709401709, 'recall': 0.6041666666666666, 'f1-score': 0.5613275613275612, 'support': 48}}
*************************************************
*************************************************
----------------------------------------
Stage -  train
0
1000000
1
0.06666666666666667
2
0.0

In [30]:
to_dump = {'clf_report':ITERATION_CLF_REPORT, 'file_change':ITERATION_FILE_CHANGE, 'ldc_clfs':LDC_CLFS, 'gmms': trained_GMMs}

In [31]:
with open(WRITE_PATH+EXPERIMENT_TYPE+'.pickle', 'wb') as handle:
    pickle.dump(to_dump, handle, protocol=pickle.HIGHEST_PROTOCOL)

# For unnormalised

In [35]:
EXPERIMENT_TYPE = 'ldc_0.5'

In [36]:
	# for every epoch
epoch = 0

ITERATION_CLF_REPORT = []
ITERATION_FILE_CHANGE = []
LDC_CLFS = []


for iterations in range(400):

    sampled_df = stratified_sample_df(temp_df)

    train_df, test_df = train_test_split(sampled_df, test_size = 0.33, stratify = sampled_df[['speaker','speech_type']])

    train_df = get_audio_features(train_df)
    test_df = get_audio_features(test_df)

    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)


    # for 400 iterations
    ldc_clf = None
    epsilon = 1000000
    max_iters = 1000

    ldc_clf = LinearDiscriminantAnalysis(solver='lsqr')

    print('*************************************************') 
    print('ITERATION NUMBER - ',iterations)
    print('*************************************************') 


    for stage in ['train', 'test']:

        print('----------------------------------------')   
        print('Stage - ', stage)


        if stage == 'train':

            ITERATION_FILE_CHANGE_ELEM = []

            train_df['inferred'] = train_df.apply(infer_GMM,axis=1)

            ldc_clf.fit(np.array(train_df['inferred'].tolist()),train_df['speech_type'].values)

            train_df['predicted_likelihood'] = ldc_clf.predict_proba(np.array(train_df['inferred'].tolist()))[:,0]

            train_df, epsilon = get_stopping_criteria(train_df, 0)

            print(epsilon)
            ITERATION_FILE_CHANGE_ELEM.append(epsilon)
            LDC_CLFS.append(ldc_clf)

            train_df['prev_changed_speech_type'] = train_df['changed_speech_type']

            ITERATION_FILE_CHANGE.append(ITERATION_FILE_CHANGE_ELEM)

        else:

            sampled_df_test = test_df

            sampled_df_test['inferred'] = sampled_df_test.apply(infer_GMM,axis=1)

            sampled_df_test['predicted_likelihood'] = ldc_clf.predict_proba(np.array(sampled_df_test['inferred'].tolist()))[:,0]

            sampled_df_test = get_pred_labels(sampled_df_test)

            clf_report = classification_report(sampled_df_test['speech_type'],sampled_df_test['changed_speech_type'], output_dict=True)
            ITERATION_CLF_REPORT.append(clf_report)
            print(clf_report)

*************************************************
ITERATION NUMBER -  0
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6388888888888888, 'recall': 0.9583333333333334, 'f1-score': 0.7666666666666666, 'support': 24}, '1': {'precision': 0.9166666666666666, 'recall': 0.4583333333333333, 'f1-score': 0.611111111111111, 'support': 24}, 'accuracy': 0.7083333333333334, 'macro avg': {'precision': 0.7777777777777777, 'recall': 0.7083333333333334, 'f1-score': 0.6888888888888889, 'support': 48}, 'weighted avg': {'precision': 0.7777777777777777, 'recall': 0.7083333333333334, 'f1-score': 0.6888888888888888, 'support': 48}}
*************************************************
ITERATION NUMBER -  1
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test

*************************************************
ITERATION NUMBER -  12
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.5833333333333334, 'recall': 0.875, 'f1-score': 0.7000000000000001, 'support': 24}, '1': {'precision': 0.75, 'recall': 0.375, 'f1-score': 0.5, 'support': 24}, 'accuracy': 0.625, 'macro avg': {'precision': 0.6666666666666667, 'recall': 0.625, 'f1-score': 0.6000000000000001, 'support': 48}, 'weighted avg': {'precision': 0.6666666666666666, 'recall': 0.625, 'f1-score': 0.6, 'support': 48}}
*************************************************
ITERATION NUMBER -  13
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6764705882352942, 'recall': 0.9583333333333334, 'f1-score': 0.7931034482758621, 'su

*************************************************
ITERATION NUMBER -  24
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6111111111111112, 'recall': 0.9166666666666666, 'f1-score': 0.7333333333333334, 'support': 24}, '1': {'precision': 0.8333333333333334, 'recall': 0.4166666666666667, 'f1-score': 0.5555555555555556, 'support': 24}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.7222222222222223, 'recall': 0.6666666666666666, 'f1-score': 0.6444444444444445, 'support': 48}, 'weighted avg': {'precision': 0.7222222222222223, 'recall': 0.6666666666666666, 'f1-score': 0.6444444444444445, 'support': 48}}
*************************************************
ITERATION NUMBER -  25
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  t

{'0': {'precision': 0.5853658536585366, 'recall': 1.0, 'f1-score': 0.7384615384615384, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.2916666666666667, 'f1-score': 0.45161290322580644, 'support': 24}, 'accuracy': 0.6458333333333334, 'macro avg': {'precision': 0.7926829268292683, 'recall': 0.6458333333333334, 'f1-score': 0.5950372208436724, 'support': 48}, 'weighted avg': {'precision': 0.7926829268292682, 'recall': 0.6458333333333334, 'f1-score': 0.5950372208436724, 'support': 48}}
*************************************************
ITERATION NUMBER -  36
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6388888888888888, 'recall': 0.9583333333333334, 'f1-score': 0.7666666666666666, 'support': 24}, '1': {'precision': 0.9166666666666666, 'recall': 0.4583333333333333, 'f1-score': 0.611111111111111, 'support': 24}, 'accuracy': 0.7083333333333334, '

*************************************************
ITERATION NUMBER -  47
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6486486486486487, 'recall': 1.0, 'f1-score': 0.7868852459016393, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.4583333333333333, 'f1-score': 0.6285714285714286, 'support': 24}, 'accuracy': 0.7291666666666666, 'macro avg': {'precision': 0.8243243243243243, 'recall': 0.7291666666666666, 'f1-score': 0.7077283372365339, 'support': 48}, 'weighted avg': {'precision': 0.8243243243243242, 'recall': 0.7291666666666666, 'f1-score': 0.7077283372365338, 'support': 48}}
*************************************************
ITERATION NUMBER -  48
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6666

...

*************************************************
ITERATION NUMBER -  345
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6153846153846154, 'recall': 1.0, 'f1-score': 0.761904761904762, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.375, 'f1-score': 0.5454545454545454, 'support': 24}, 'accuracy': 0.6875, 'macro avg': {'precision': 0.8076923076923077, 'recall': 0.6875, 'f1-score': 0.6536796536796536, 'support': 48}, 'weighted avg': {'precision': 0.8076923076923078, 'recall': 0.6875, 'f1-score': 0.6536796536796537, 'support': 48}}
*************************************************
ITERATION NUMBER -  346
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.5609756097560976, 'recall': 0.9583333333333334, 'f1-

*************************************************
ITERATION NUMBER -  357
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.5714285714285714, 'recall': 1.0, 'f1-score': 0.7272727272727273, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.25, 'f1-score': 0.4, 'support': 24}, 'accuracy': 0.625, 'macro avg': {'precision': 0.7857142857142857, 'recall': 0.625, 'f1-score': 0.5636363636363637, 'support': 48}, 'weighted avg': {'precision': 0.7857142857142857, 'recall': 0.625, 'f1-score': 0.5636363636363636, 'support': 48}}
*************************************************
ITERATION NUMBER -  358
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6857142857142857, 'recall': 1.0, 'f1-score': 0.8135593220338984, 'supp

{'0': {'precision': 0.6285714285714286, 'recall': 0.9166666666666666, 'f1-score': 0.7457627118644068, 'support': 24}, '1': {'precision': 0.8461538461538461, 'recall': 0.4583333333333333, 'f1-score': 0.5945945945945945, 'support': 24}, 'accuracy': 0.6875, 'macro avg': {'precision': 0.7373626373626374, 'recall': 0.6875, 'f1-score': 0.6701786532295007, 'support': 48}, 'weighted avg': {'precision': 0.7373626373626374, 'recall': 0.6875, 'f1-score': 0.6701786532295007, 'support': 48}}
*************************************************
ITERATION NUMBER -  369
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.575, 'recall': 0.9583333333333334, 'f1-score': 0.71875, 'support': 24}, '1': {'precision': 0.875, 'recall': 0.2916666666666667, 'f1-score': 0.43750000000000006, 'support': 24}, 'accuracy': 0.625, 'macro avg': {'precision': 0.725, 'recall': 0.625, 'f1-

*************************************************
ITERATION NUMBER -  380
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.5853658536585366, 'recall': 1.0, 'f1-score': 0.7384615384615384, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.2916666666666667, 'f1-score': 0.45161290322580644, 'support': 24}, 'accuracy': 0.6458333333333334, 'macro avg': {'precision': 0.7926829268292683, 'recall': 0.6458333333333334, 'f1-score': 0.5950372208436724, 'support': 48}, 'weighted avg': {'precision': 0.7926829268292682, 'recall': 0.6458333333333334, 'f1-score': 0.5950372208436724, 'support': 48}}
*************************************************
ITERATION NUMBER -  381
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.6

*************************************************
ITERATION NUMBER -  392
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0.5454545454545454, 'recall': 1.0, 'f1-score': 0.7058823529411764, 'support': 24}, '1': {'precision': 1.0, 'recall': 0.16666666666666666, 'f1-score': 0.2857142857142857, 'support': 24}, 'accuracy': 0.5833333333333334, 'macro avg': {'precision': 0.7727272727272727, 'recall': 0.5833333333333334, 'f1-score': 0.49579831932773105, 'support': 48}, 'weighted avg': {'precision': 0.7727272727272728, 'recall': 0.5833333333333334, 'f1-score': 0.49579831932773105, 'support': 48}}
*************************************************
ITERATION NUMBER -  393
*************************************************
----------------------------------------
Stage -  train
1000000
----------------------------------------
Stage -  test
{'0': {'precision': 0

In [37]:
to_dump = {'clf_report':ITERATION_CLF_REPORT, 'file_change':ITERATION_FILE_CHANGE, 'ldc_clfs':LDC_CLFS, 'gmms': trained_GMMs}
with open(WRITE_PATH+EXPERIMENT_TYPE+'.pickle', 'wb') as handle:
    pickle.dump(to_dump, handle, protocol=pickle.HIGHEST_PROTOCOL)